<a href="https://colab.research.google.com/github/Mcuong17/DoanWEB-demo/blob/main/Airline_Time_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Giới thiệu topic
Đối mặt với sự chậm trễ của các chuyến bay luôn là một điều rất phiền toái với khách hang bởi vì điều đó là điều họ không thể dự đoán hay lường trước rằng việc delay chuyến bay sẽ xảy ra. Việc chậm trễ khi bay có thể dẫn đến lãng phí thời gian và có thể làm xáo trộn lịch bay của chuyến bay tiếp theo. Do đó bài phân tích này nhằm mục đích kiểm tra các dữ liệu của các chuyến bay và tìm hiểu các dạng chậm trễ của chuyến bay, giúp khách hang có thể dự đoán được sự chậm trễ đó xảy ra như thế nào.

# Data Pre Processing( Tiền xử lý dữ liệu)
Đầu tiên chúng ta sẽ thêm các thư viện cần dùng
Những thư viện được sử dụng đó là:
1.   Numpy: được sử dụng cho các hàm tính toán, xử lý các mảng dữ liệu numberic
2.   Pandas: Được sử dụng để xử lý các vấn đề liên quan đến dữ liệu, dataset...
3.    Matplotlib và Seaborn: Được sử dụng cho việc trực quan hóa dữ liệu
4.    Sklearn: Tạo mô hình và chứa một số chức năng tiền xử lý dữ liệu hữu ích



In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Load dataframe

In [10]:

df1 = pd.read_csv("1999.csv")
df1.head(5)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,1999,1,27,3,1906.0,1908,2024.0,2005,US,1244,...,3,9,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,1999,1,28,4,2016.0,1908,2126.0,2005,US,1244,...,19,10,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,1999,1,29,5,1907.0,1908,2000.0,2005,US,1244,...,2,8,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,1999,1,31,7,1932.0,1908,2031.0,2005,US,1244,...,6,8,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,1999,1,1,5,1601.0,1535,1707.0,1645,US,297,...,11,8,0,NaN,0,NaN,NaN,NaN,NaN,NaN


In [6]:
df2 = pd.read_csv("2000.csv")

In [8]:
df = pd.concat([df1, df2])

## Tổng quan về dataframe, tên các cột và kích thước của df

In [9]:
df.columns

Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay',
       'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay'],
      dtype='object')

Dựa vào dữ liệu đã cung cấp: Dataset bao gồm hơn 11 triệu observation về chuyến bay với 29 cột:
*   **Year [int]:** Bao gồm thông tin về năm(1999&2000)
*   **Month [int]:** Thông tin về tháng(1 - Jan, 2 - Feb..)
*   **DayofMonth [int]**: Ngày trong tháng(1-31, nếu có)
*   **DayOfWeek [int]**: Ngày trong tuần(1 - Mon, 2 - Tue..)
*   **DepTime [int]**: Thời gian khởi hành thực tế(theo múi giờ địa phương)
*   **CRSDepTime [int]**: Thời gian khởi hành theo lịch trình(theo múi giờ địa phương)
*   **ArrTime [int]**: Thời gian thực tế(theo múi giờ địa phương)
*   **UniqueCarrier [int]**: Mã nhà cung cấp dịch vụ để xác định các nhà cung cấp dịch vụ trong file carriers.csv
*   **FlightNum [int]**: Số chuyến bay
*   **TailNum [str]**: Số hiệu trên đuôi máy bay có trong file plane-data.csv
*   **ActualElapsedTime [int]**: Sự khác biệt giữa ArrTime và DepTime trong vài phút, đồng thời cũng là tổng của AirTime, TaxiIn, TaxiOut
*   **CRSElapsedTime [int]**:  Sự khác biệt giữa CRSArrTime và CRSDepTime trong vài phút
*   ** ArrDelay [int]:**Sự khác biệt giữa ArrTime và CRSArrTime trong vài phút
*    **DepDelay [int]**: Sự khác biệt giữa DepTime and CRSDepTime trong vài phút
*    **Origin [str]**: Sử dụng để xác định mã sân bay IATA mà chuyến bay đã khởi hành trong file airports.csv
*    **Dest [str]**: Sử dụng để xác định mã sân bay IATA mà chuyến bay đã khởi hành trong file airports.csv
*    **Distance [int]**: Khoảng cách bay( tính bằng dặm)
*    **TaxiIn [int]**: Thời gian taxi đến tính bằng phút
*    **TaxiOut [int]**: Thời gian taxi trả tính bằng phút
*    **Cancelled [int]**: Hủy chuyến bay(1 - bị hủy, 0 - Không bị hủy)
*    **CancellationCode [str]**: Lý do hủy chuyến bay ( A - Hãng vận chuyển, B - Thời tiết, C - Hệ thống hàng không quốc gia, D - An ninh)
*   ** Diverted [int]**: chuyển hướng( 1 - Chuyển hướng, 0 - không chuyển hướng)
*   **WeatherDelay [int]**: Sự chậm trễ ảnh hưởng do thời tiết tính bằng phút
*   **NASDelay [int]**: Sự chậm trễ ảnh hưởng do hệ thống hàng không quốc gia gây ra trong vài phút
*  ** LateAircraftDelay [int]**: Sự chậm trể ảnh hưởng bởi chuyến bay trước cũng bị muộn

-  Dựa trên các cột, có 3 taajo dữ liệu bổ sung cần thêm vào như sau:
*   **airports.csv**: Chứa thông tin về các chuyến bay khởi hành và chuyến bay đến
*   **carriers.csv:** Chứa thông tin về hãng bay
*   **plane-data.csv**: Chứa thông tin về máy bay


In [14]:
airport_df = pd.read_csv("airports.csv")
cariers_df = pd.read_csv("carriers.csv")
plane_df = pd.read_csv("plane-data.csv")

  Tên cột và hình dạng bảng dữ liệu

In [15]:
airport_df.columns, airport_df.shape

(Index(['iata', 'airport', 'city', 'state', 'country', 'lat', 'long'], dtype='object'),
 (3376, 7))